# 🔬 胚胎著床預測 - Baseline 模型訓練

> EXP-001: 使用 ResNet18 建立基準模型

**目標**：建立第一個基準模型，了解資料特性與基本效能

---

## 1️⃣ 環境設置

In [ ]:
# 掛載 Google Drive（如果在 Colab 上執行）
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive 已掛載")
else:
    print("ℹ️ 不在 Colab 環境，跳過掛載")

In [ ]:
# 安裝必要套件（Colab 通常已預裝）
# !pip install torch torchvision scikit-learn matplotlib seaborn tqdm

In [ ]:
# 匯入套件
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, roc_auc_score, precision_score, 
    recall_score, f1_score, confusion_matrix, classification_report
)

# 設定隨機種子（確保可重現）
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# 檢查 GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ 使用裝置: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

## 2️⃣ 設定資料路徑

⚠️ **請根據你的資料位置修改下方路徑**

In [ ]:
# =============================================
# ⚠️ 請修改這裡的路徑！
# =============================================

if IN_COLAB:
    # Colab：資料在 Google Drive 中
    DATA_ROOT = '/content/drive/MyDrive/hvwc23'  # ← 修改成你的路徑
else:
    # 本機
    DATA_ROOT = '../data/raw/hvwc23'

# 確認路徑
TRAIN_DIR = os.path.join(DATA_ROOT, 'train')
TEST_DIR = os.path.join(DATA_ROOT, 'test')
TRAIN_CSV = os.path.join(DATA_ROOT, 'train.csv')
TEST_CSV = os.path.join(DATA_ROOT, 'test.csv')

# 檢查檔案是否存在
print("📁 檢查資料路徑...")
print(f"   訓練資料夾: {os.path.exists(TRAIN_DIR)} - {TRAIN_DIR}")
print(f"   測試資料夾: {os.path.exists(TEST_DIR)} - {TEST_DIR}")
print(f"   訓練 CSV:   {os.path.exists(TRAIN_CSV)} - {TRAIN_CSV}")

## 3️⃣ 載入與探索資料

In [ ]:
# 載入標籤
train_df = pd.read_csv(TRAIN_CSV)
print(f"📊 訓練資料筆數: {len(train_df)}")
print()
train_df.head()

In [ ]:
# 類別分佈
print("📊 類別分佈:")
class_counts = train_df['Class'].value_counts().sort_index()
for cls, count in class_counts.items():
    pct = count / len(train_df) * 100
    label = "不會著床" if cls == 0 else "會著床"
    print(f"   Class {cls} ({label}): {count} 張 ({pct:.1f}%)")

# 視覺化
plt.figure(figsize=(8, 4))
colors = ['#ff6b6b', '#4ecdc4']
plt.bar(['Class 0\n(不會著床)', 'Class 1\n(會著床)'], class_counts.values, color=colors)
plt.title('類別分佈（嚴重不平衡！）', fontsize=14)
plt.ylabel('數量')
for i, v in enumerate(class_counts.values):
    plt.text(i, v + 10, f'{v}', ha='center', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# 查看一些樣本圖片
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

# Class 0 樣本
class0_samples = train_df[train_df['Class'] == 0].sample(5)
for i, (_, row) in enumerate(class0_samples.iterrows()):
    img_path = os.path.join(TRAIN_DIR, row['Image'])
    img = Image.open(img_path)
    axes[0, i].imshow(img)
    axes[0, i].set_title(f"Class 0\n{row['Image']}", fontsize=9)
    axes[0, i].axis('off')

# Class 1 樣本
class1_samples = train_df[train_df['Class'] == 1].sample(5)
for i, (_, row) in enumerate(class1_samples.iterrows()):
    img_path = os.path.join(TRAIN_DIR, row['Image'])
    img = Image.open(img_path)
    axes[1, i].imshow(img)
    axes[1, i].set_title(f"Class 1\n{row['Image']}", fontsize=9)
    axes[1, i].axis('off')

plt.suptitle('樣本圖片（上排: 不會著床, 下排: 會著床）', fontsize=14)
plt.tight_layout()
plt.show()

## 4️⃣ 資料前處理與分割

In [ ]:
# 分割訓練集和驗證集（80/20，分層抽樣）
train_data, val_data = train_test_split(
    train_df, 
    test_size=0.2, 
    random_state=SEED, 
    stratify=train_df['Class']  # 確保類別比例一致
)

print(f"📊 資料分割結果:")
print(f"   訓練集: {len(train_data)} 張")
print(f"   驗證集: {len(val_data)} 張")
print()
print(f"   訓練集類別分佈: {dict(train_data['Class'].value_counts().sort_index())}")
print(f"   驗證集類別分佈: {dict(val_data['Class'].value_counts().sort_index())}")

In [ ]:
# 定義資料增強與轉換
IMG_SIZE = 224  # ResNet 標準輸入大小

# 訓練集：有資料增強
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet 標準
        std=[0.229, 0.224, 0.225]
    )
])

# 驗證集：不做增強
val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

print("✅ 資料轉換定義完成")

In [ ]:
# 自定義 Dataset
class EmbryoDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        img_path = os.path.join(self.img_dir, row['Image'])
        image = Image.open(img_path).convert('RGB')
        label = row['Class']
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# 建立 Dataset
train_dataset = EmbryoDataset(train_data, TRAIN_DIR, train_transform)
val_dataset = EmbryoDataset(val_data, TRAIN_DIR, val_transform)

print(f"✅ Dataset 建立完成")
print(f"   訓練集: {len(train_dataset)} 張")
print(f"   驗證集: {len(val_dataset)} 張")

In [ ]:
# 建立 DataLoader
BATCH_SIZE = 32

train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"✅ DataLoader 建立完成")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   訓練 batches: {len(train_loader)}")
print(f"   驗證 batches: {len(val_loader)}")

## 5️⃣ 建立模型

In [ ]:
# 使用預訓練的 ResNet18
model = models.resnet18(weights='IMAGENET1K_V1')

# 修改最後一層（原本是 1000 類，改成 2 類）
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

# 移到 GPU
model = model.to(device)

print("✅ 模型建立完成: ResNet18")
print(f"   參數量: {sum(p.numel() for p in model.parameters()):,}")
print(f"   可訓練參數: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

In [ ]:
# 計算類別權重（處理不平衡）
class_counts = train_data['Class'].value_counts().sort_index()
total = len(train_data)
class_weights = torch.tensor([total / (2 * class_counts[i]) for i in range(2)], dtype=torch.float32)
class_weights = class_weights.to(device)

print(f"📊 類別權重（用於處理不平衡）:")
print(f"   Class 0: {class_weights[0]:.4f}")
print(f"   Class 1: {class_weights[1]:.4f}")

In [ ]:
# 定義損失函數和優化器
criterion = nn.CrossEntropyLoss(weight=class_weights)  # 加權損失
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

print("✅ 訓練設定完成")
print(f"   損失函數: CrossEntropyLoss (加權)")
print(f"   優化器: Adam (lr=0.001)")
print(f"   學習率調度: StepLR (每 10 epoch 降為 0.1 倍)")

## 6️⃣ 訓練模型

In [ ]:
# 訓練函數
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(loader), correct / total

# 驗證函數
def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_probs = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            probs = torch.softmax(outputs, dim=1)
            _, predicted = outputs.max(1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())  # Class 1 的機率
            all_labels.extend(labels.cpu().numpy())
    
    # 計算指標
    accuracy = accuracy_score(all_labels, all_preds)
    
    # 處理 AUC（如果只有一個類別會報錯）
    try:
        auc = roc_auc_score(all_labels, all_probs)
    except:
        auc = 0.0
    
    return running_loss / len(loader), accuracy, auc, all_preds, all_probs, all_labels

print("✅ 訓練函數定義完成")

In [ ]:
# 開始訓練
NUM_EPOCHS = 30

# 記錄歷史
history = {
    'train_loss': [], 'train_acc': [],
    'val_loss': [], 'val_acc': [], 'val_auc': []
}

best_auc = 0.0
best_epoch = 0

print("="*60)
print("🚀 開始訓練")
print("="*60)

for epoch in range(NUM_EPOCHS):
    # 訓練
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # 驗證
    val_loss, val_acc, val_auc, _, _, _ = validate(model, val_loader, criterion, device)
    
    # 更新學習率
    scheduler.step()
    
    # 記錄
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['val_auc'].append(val_auc)
    
    # 儲存最佳模型
    if val_auc > best_auc:
        best_auc = val_auc
        best_epoch = epoch + 1
        torch.save(model.state_dict(), 'best_model.pth')
    
    # 顯示進度
    print(f"Epoch {epoch+1:02d}/{NUM_EPOCHS} | "
          f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, AUC: {val_auc:.4f}")

print("="*60)
print(f"✅ 訓練完成！最佳 AUC: {best_auc:.4f} (Epoch {best_epoch})")
print("="*60)

## 7️⃣ 視覺化訓練過程

In [ ]:
# 繪製訓練曲線
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Loss
axes[0].plot(history['train_loss'], label='Train', marker='o', markersize=3)
axes[0].plot(history['val_loss'], label='Validation', marker='o', markersize=3)
axes[0].set_title('Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history['train_acc'], label='Train', marker='o', markersize=3)
axes[1].plot(history['val_acc'], label='Validation', marker='o', markersize=3)
axes[1].set_title('Accuracy')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# AUC
axes[2].plot(history['val_auc'], label='Validation AUC', marker='o', markersize=3, color='green')
axes[2].axhline(y=best_auc, color='r', linestyle='--', label=f'Best: {best_auc:.4f}')
axes[2].set_title('Validation AUC')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('AUC')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_curves.png', dpi=150)
plt.show()
print("📊 訓練曲線已儲存: training_curves.png")

## 8️⃣ 最終評估

In [ ]:
# 載入最佳模型
model.load_state_dict(torch.load('best_model.pth'))
print(f"✅ 載入最佳模型 (Epoch {best_epoch})")

# 最終驗證
val_loss, val_acc, val_auc, all_preds, all_probs, all_labels = validate(
    model, val_loader, criterion, device
)

In [ ]:
# 計算詳細指標
print("="*60)
print("📊 最終評估結果")
print("="*60)
print()
print(f"✅ Accuracy:  {val_acc:.4f} ({val_acc*100:.1f}%)")
print(f"✅ AUC-ROC:   {val_auc:.4f}")
print()

# 各類別指標
precision_0 = precision_score(all_labels, all_preds, pos_label=0)
precision_1 = precision_score(all_labels, all_preds, pos_label=1)
recall_0 = recall_score(all_labels, all_preds, pos_label=0)
recall_1 = recall_score(all_labels, all_preds, pos_label=1)
f1_0 = f1_score(all_labels, all_preds, pos_label=0)
f1_1 = f1_score(all_labels, all_preds, pos_label=1)

print("📊 各類別指標:")
print(f"{'':15} {'Precision':>10} {'Recall':>10} {'F1-Score':>10}")
print("-" * 50)
print(f"{'Class 0 (不著床)':15} {precision_0:>10.4f} {recall_0:>10.4f} {f1_0:>10.4f}")
print(f"{'Class 1 (著床)':15} {precision_1:>10.4f} {recall_1:>10.4f} {f1_1:>10.4f}")
print()

# ⚠️ 重點指標
print("⚠️ 重點關注 Class 1 (會著床) 的 Recall!")
print(f"   → 會著床的胚胎，模型猜對了 {recall_1*100:.1f}%")

In [ ]:
# 混淆矩陣
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['預測: 不著床', '預測: 著床'],
            yticklabels=['實際: 不著床', '實際: 著床'])
plt.title('混淆矩陣 (Confusion Matrix)')
plt.ylabel('實際值')
plt.xlabel('預測值')

# 標註說明
tn, fp, fn, tp = cm.ravel()
plt.figtext(0.5, -0.05, 
            f'TN={tn} (猜對不著床) | FP={fp} (誤報) | FN={fn} (漏掉) | TP={tp} (猜對著床)', 
            ha='center', fontsize=10)

plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()
print("📊 混淆矩陣已儲存: confusion_matrix.png")

In [ ]:
# 完整分類報告
print("\n📊 完整分類報告:")
print(classification_report(all_labels, all_preds, target_names=['不著床', '著床']))

## 9️⃣ 結果摘要

In [ ]:
# 產生結果摘要（方便複製回報）
print("="*60)
print("📋 EXP-001 結果摘要（請複製此區塊回報）")
print("="*60)
print(f"""
## 實驗設定
- 模型: ResNet18 (預訓練)
- 訓練集: {len(train_data)} 張
- 驗證集: {len(val_data)} 張
- Batch size: {BATCH_SIZE}
- Epochs: {NUM_EPOCHS}
- 類別加權: 是

## 結果
| 指標 | 數值 |
|------|------|
| Accuracy | {val_acc:.4f} ({val_acc*100:.1f}%) |
| AUC-ROC | {val_auc:.4f} |
| Precision (著床) | {precision_1:.4f} |
| Recall (著床) | {recall_1:.4f} |
| F1-Score (著床) | {f1_1:.4f} |

## 混淆矩陣
- TN (猜對不著床): {tn}
- FP (誤報): {fp}
- FN (漏掉): {fn}
- TP (猜對著床): {tp}

## 觀察
- (請填寫你的觀察)
""")

---

## 🎯 下一步建議

根據結果，可以嘗試：

1. **如果 Recall 太低**：
   - 調整判斷閾值（threshold）
   - 增加 Class 1 的權重
   - 嘗試 Oversampling

2. **如果過擬合**（訓練好但驗證差）：
   - 增加 Dropout
   - 增加資料增強
   - 減少模型複雜度

3. **如果整體效果不好**：
   - 嘗試其他模型（EfficientNet, ConvNeXt）
   - 增加訓練 epochs
   - 調整學習率